## key points:
### 1.Preprocessing: flatten inputs and labels need one-hot encoding
### 2.Batch training: SGD to accelerate training rate
### 3.Add one hidden layer to improve accuracy
### 4.Problems:
#### —— Regularization: loss function add L2_loss    
#### ——Overfitting: use less batch data to trainning model  
#### ——Dropout: close some neurons with keep_probability

In [ ]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

#### import data

In [21]:
pickle_file = r'D:\exercise\dataset\notMNIST\notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save
    print('Training set',train_dataset.shape, train_labels.shape)
    print('Validation set',valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


#### images_x flatten & lables_y one-hot encoding

In [24]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    #flatten
    dataset = dataset.reshape((-1, image_size*image_size)).astype(np.float32) 
    #one-hot
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32) #labels[:,None]给array增加一个维度,[[1],[2]...]
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


#### select part of train_dataset for training

In [25]:
train_subset = 10000

graph = tf.Graph()
with graph.as_default():
    
    #input data
    tf_train_dataset = tf.constant(train_dataset[:train_subset,:])
    tf_train_labels = tf.constant(train_labels[:train_subset,:])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #variables
    #想求解的参数——weights_matrix
    #需要随机赋予初始值
    #biases初始化为0
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    #training computation
    #logit
    #loss
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels,logits = logits))
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [18]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions,1) == np.argmax(labels,1)) / predictions.shape[0])

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        _, l ,predictions = sess.run([optimizer,loss,train_prediction])
        if step%100 == 0:
            print('Loss at step %d:%f'%(step,l))
            print('Training accuracy:%.1f%%'%accuracy(
                predictions, train_labels[:train_subset,:]))
            print('Validation accuracy:%.1f%%'%accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy:%.1f%%'%accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0:16.271410
Training accuracy:13.3%
Validation accuracy:15.3%
Loss at step 100:2.367983
Training accuracy:71.5%
Validation accuracy:70.2%
Loss at step 200:1.875045
Training accuracy:74.6%
Validation accuracy:72.6%
Loss at step 300:1.615165
Training accuracy:76.2%
Validation accuracy:73.4%
Loss at step 400:1.444160
Training accuracy:77.5%
Validation accuracy:74.2%
Loss at step 500:1.318590
Training accuracy:78.1%
Validation accuracy:74.5%
Loss at step 600:1.220806
Training accuracy:78.6%
Validation accuracy:74.5%
Loss at step 700:1.141978
Training accuracy:79.1%
Validation accuracy:74.6%
Loss at step 800:1.076775
Training accuracy:79.5%
Validation accuracy:74.8%
Test accuracy:82.3%


#### batch training

In [41]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    #input data
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #variable
    weights = tf.Variable(tf.truncated_normal([image_size*image_size,num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    #training
    logits = tf.matmul(tf_train_dataset,weights) + biases
    loss = tf.reduce_mean(
         tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf_test_dataset, weights) + biases)   

In [42]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step*batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels: batch_labels}
        _,l,predictions = session.run(
            [optimizer, loss, train_prediction],feed_dict = feed_dict)
        if step % 500 == 0:
            print("Minibatch loss at step %d：%f"%(step,l))
            print("Minibatch accuracy：%.1f%%"%accuracy(predictions,batch_labels))
            print("Validation accuracy：%.1f%%"%accuracy(valid_prediction.eval(),valid_labels))
            print("Test accuracy：%.1f%%"%accuracy(test_prediction.eval(),test_labels))
        



Initialized
Minibatch loss at step 0：16.535225
Minibatch accuracy：8.6%
Validation accuracy：16.1%
Test accuracy：16.9%
Minibatch loss at step 500：1.391599
Minibatch accuracy：77.3%
Validation accuracy：75.8%
Test accuracy：82.7%
Minibatch loss at step 1000：1.585522
Minibatch accuracy：78.1%
Validation accuracy：76.6%
Test accuracy：83.4%
Minibatch loss at step 1500：0.805821
Minibatch accuracy：81.2%
Validation accuracy：77.0%
Test accuracy：83.9%
Minibatch loss at step 2000：0.921110
Minibatch accuracy：82.0%
Validation accuracy：77.8%
Test accuracy：84.7%
Minibatch loss at step 2500：1.228583
Minibatch accuracy：73.4%
Validation accuracy：78.4%
Test accuracy：85.7%
Minibatch loss at step 3000：0.993225
Minibatch accuracy：79.7%
Validation accuracy：78.8%
Test accuracy：85.8%


#### add one-hidden-layer

In [77]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    #input data
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #variable
    weights_1 = tf.Variable(tf.truncated_normal([image_size*image_size,1024]))  #1024 nodes
    biases_1 = tf.Variable(tf.zeros([1024]))      
    weights_2 = tf.Variable(tf.truncated_normal([1024,num_labels])) #num_labels=10(output_nums)
    biases_2 = tf.Variable(tf.zeros([num_labels])+0.1)
    
       
    #1-hidden layer
    hidden_relu = tf.add(tf.matmul(tf_train_dataset,weights_1), biases_1)
    hidden_relu = tf.nn.relu(hidden_relu)
    logits = tf.matmul(hidden_relu, weights_2) + biases_2
    
    
    #loss
    loss = tf.reduce_mean(
         tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    

    #predictions
    train_prediction = tf.nn.softmax(logits)
    valid_hidden_relu = tf.nn.relu(tf.add(tf.matmul(tf_valid_dataset,weights_1), biases_1))
    valid_prediction = tf.nn.softmax(
        tf.matmul(valid_hidden_relu, weights_2) + biases_2) 
    test_hidden_relu = tf.nn.relu(tf.add(tf.matmul(tf_test_dataset,weights_1), biases_1))
    test_prediction = tf.nn.softmax(
        tf.matmul(test_hidden_relu, weights_2) + biases_2)

In [78]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step*batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels: batch_labels}
        _,l,predictions = session.run(
            [optimizer, loss, train_prediction],feed_dict = feed_dict)
        if step % 500 == 0:
            print("Minibatch loss at step %d：%f"%(step,l))
            print("Minibatch accuracy：%.1f%%"%accuracy(predictions,batch_labels))
            print("Validation accuracy：%.1f%%"%accuracy(valid_prediction.eval(),valid_labels))
            print("Test accuracy：%.1f%%"%accuracy(test_prediction.eval(),test_labels))
        

Initialized
Minibatch loss at step 0：357.191284
Minibatch accuracy：7.8%
Validation accuracy：25.8%
Test accuracy：27.3%
Minibatch loss at step 500：12.543827
Minibatch accuracy：85.2%
Validation accuracy：80.6%
Test accuracy：87.4%
Minibatch loss at step 1000：12.308599
Minibatch accuracy：78.1%
Validation accuracy：80.1%
Test accuracy：87.6%
Minibatch loss at step 1500：8.177224
Minibatch accuracy：86.7%
Validation accuracy：81.3%
Test accuracy：88.2%
Minibatch loss at step 2000：2.789202
Minibatch accuracy：86.7%
Validation accuracy：82.5%
Test accuracy：89.1%
Minibatch loss at step 2500：2.664767
Minibatch accuracy：84.4%
Validation accuracy：81.9%
Test accuracy：88.8%
Minibatch loss at step 3000：1.330249
Minibatch accuracy：85.9%
Validation accuracy：82.0%
Test accuracy：89.1%


### Problem 1: Regularization

In [92]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    #input data
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #variable
    weights_1 = tf.Variable(tf.truncated_normal([image_size*image_size,1024]))  #1024 nodes
    biases_1 = tf.Variable(tf.zeros([1024]))      
    weights_2 = tf.Variable(tf.truncated_normal([1024,num_labels])) #num_labels=10(output_nums)
    biases_2 = tf.Variable(tf.zeros([num_labels])+0.1)
    
       
    #1-hidden layer
    hidden_relu = tf.add(tf.matmul(tf_train_dataset,weights_1), biases_1)
    hidden_relu = tf.nn.relu(hidden_relu)
    logits = tf.matmul(hidden_relu, weights_2) + biases_2
    
    
    #loss
    l2_loss = tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_1)
    loss = tf.reduce_mean(
         tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + 0.001*l2_loss #train_loss控制在同一数量级
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    

    #predictions
    train_prediction = tf.nn.softmax(logits)
    valid_hidden_relu = tf.nn.relu(tf.add(tf.matmul(tf_valid_dataset,weights_1), biases_1))
    valid_prediction = tf.nn.softmax(
        tf.matmul(valid_hidden_relu, weights_2) + biases_2) 
    test_hidden_relu = tf.nn.relu(tf.add(tf.matmul(tf_test_dataset,weights_1), biases_1))
    test_prediction = tf.nn.softmax(
        tf.matmul(test_hidden_relu, weights_2) + biases_2)

In [93]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step*batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels: batch_labels}
        _,l,predictions = session.run(
            [optimizer, loss, train_prediction],feed_dict = feed_dict)
        if step % 500 == 0:
            print("Minibatch loss at step %d：%f"%(step,l))
            print("Minibatch accuracy：%.1f%%"%accuracy(predictions,batch_labels))
            print("Validation accuracy：%.1f%%"%accuracy(valid_prediction.eval(),valid_labels))
            print("Test accuracy：%.1f%%"%accuracy(test_prediction.eval(),test_labels))

Initialized
Minibatch loss at step 0：624.789551
Minibatch accuracy：10.9%
Validation accuracy：29.6%
Test accuracy：31.7%
Minibatch loss at step 500：196.430832
Minibatch accuracy：81.2%
Validation accuracy：78.8%
Test accuracy：85.2%
Minibatch loss at step 1000：115.846329
Minibatch accuracy：80.5%
Validation accuracy：81.3%
Test accuracy：88.1%
Minibatch loss at step 1500：68.570305
Minibatch accuracy：86.7%
Validation accuracy：82.7%
Test accuracy：89.9%
Minibatch loss at step 2000：41.059177
Minibatch accuracy：91.4%
Validation accuracy：84.8%
Test accuracy：91.8%
Minibatch loss at step 2500：25.143045
Minibatch accuracy：88.3%
Validation accuracy：85.5%
Test accuracy：92.5%
Minibatch loss at step 3000：15.437861
Minibatch accuracy：85.9%
Validation accuracy：86.3%
Test accuracy：92.9%


### Problem 2: overfitting

In [94]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    #input data
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #variable
    weights_1 = tf.Variable(tf.truncated_normal([image_size*image_size,1024]))  #1024 nodes
    biases_1 = tf.Variable(tf.zeros([1024]))      
    weights_2 = tf.Variable(tf.truncated_normal([1024,num_labels])) #num_labels=10(output_nums)
    biases_2 = tf.Variable(tf.zeros([num_labels])+0.1)
    
       
    #1-hidden layer
    hidden_relu = tf.add(tf.matmul(tf_train_dataset,weights_1), biases_1)
    hidden_relu = tf.nn.relu(hidden_relu)
    logits = tf.matmul(hidden_relu, weights_2) + biases_2
    
    
    #loss
    l2_loss = tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_1)
    loss = tf.reduce_mean(
         tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + 0.001*l2_loss #train_loss控制在同一数量级
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    

    #predictions
    train_prediction = tf.nn.softmax(logits)
    valid_hidden_relu = tf.nn.relu(tf.add(tf.matmul(tf_valid_dataset,weights_1), biases_1))
    valid_prediction = tf.nn.softmax(
        tf.matmul(valid_hidden_relu, weights_2) + biases_2) 
    test_hidden_relu = tf.nn.relu(tf.add(tf.matmul(tf_test_dataset,weights_1), biases_1))
    test_prediction = tf.nn.softmax(
        tf.matmul(test_hidden_relu, weights_2) + biases_2)

In [96]:
num_steps = 3001
offset_range =1000

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step*batch_size) % (offset_range)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels: batch_labels}
        _,l,predictions = session.run(
            [optimizer, loss, train_prediction],feed_dict = feed_dict)
        if step % 200 == 0:
            print("Minibatch loss at step %d：%f"%(step,l))
            print("Minibatch accuracy：%.1f%%"%accuracy(predictions,batch_labels))
            print("Validation accuracy：%.1f%%"%accuracy(valid_prediction.eval(),valid_labels))
            print("Test accuracy：%.1f%%"%accuracy(test_prediction.eval(),test_labels))

Initialized
Minibatch loss at step 0：763.342773
Minibatch accuracy：9.4%
Validation accuracy：32.3%
Test accuracy：34.9%
Minibatch loss at step 200：256.810822
Minibatch accuracy：100.0%
Validation accuracy：77.5%
Test accuracy：85.0%
Minibatch loss at step 400：210.257217
Minibatch accuracy：100.0%
Validation accuracy：77.5%
Test accuracy：85.1%
Minibatch loss at step 600：172.135437
Minibatch accuracy：100.0%
Validation accuracy：77.5%
Test accuracy：85.1%
Minibatch loss at step 800：140.925537
Minibatch accuracy：100.0%
Validation accuracy：77.4%
Test accuracy：85.1%
Minibatch loss at step 1000：115.374367
Minibatch accuracy：100.0%
Validation accuracy：77.4%
Test accuracy：85.1%
Minibatch loss at step 1200：94.455772
Minibatch accuracy：100.0%
Validation accuracy：77.4%
Test accuracy：85.1%
Minibatch loss at step 1400：77.329994
Minibatch accuracy：100.0%
Validation accuracy：77.4%
Test accuracy：85.1%
Minibatch loss at step 1600：63.309311
Minibatch accuracy：100.0%
Validation accuracy：77.4%
Test accuracy：85.1%
M

### Problem 3: Dropout

In [98]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    #input data
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #variable
    weights_1 = tf.Variable(tf.truncated_normal([image_size*image_size,1024]))  #1024 nodes
    biases_1 = tf.Variable(tf.zeros([1024]))      
    weights_2 = tf.Variable(tf.truncated_normal([1024,num_labels])) #num_labels=10(output_nums)
    biases_2 = tf.Variable(tf.zeros([num_labels])+0.1)
    
      
    #1-hidden layer
    hidden_relu = tf.add(tf.matmul(tf_train_dataset,weights_1), biases_1)
    hidden_relu = tf.nn.relu(hidden_relu)
    hidden_relu = tf.nn.dropout(hidden_relu,0.5)
    logits = tf.matmul(hidden_relu, weights_2) + biases_2
    
    
    #loss
    l2_loss = tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(weights_1)
    loss = tf.reduce_mean(
         tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + 0.001*l2_loss #train_loss控制在同一数量级
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    

    #predictions
    train_prediction = tf.nn.softmax(logits)
    valid_hidden_relu = tf.nn.relu(tf.add(tf.matmul(tf_valid_dataset,weights_1), biases_1))
    vaild_hidden_relu = tf.nn.dropout(valid_hidden_relu,0.5)
    valid_prediction = tf.nn.softmax(
        tf.matmul(valid_hidden_relu, weights_2) + biases_2) 
    test_hidden_relu = tf.nn.relu(tf.add(tf.matmul(tf_test_dataset,weights_1), biases_1))
    test_hidden_relu = tf.nn.dropout(test_hidden_relu,0.5)
    test_prediction = tf.nn.softmax(
        tf.matmul(test_hidden_relu, weights_2) + biases_2)

In [101]:
num_steps = 5001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step*batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data,
                    tf_train_labels: batch_labels}
        _,l,predictions = session.run(
            [optimizer, loss, train_prediction],feed_dict = feed_dict)
        if step % 500 == 0:
            print("Minibatch loss at step %d：%f"%(step,l))
            print("Minibatch accuracy：%.1f%%"%accuracy(predictions,batch_labels))
            print("Validation accuracy：%.1f%%"%accuracy(valid_prediction.eval(),valid_labels))
            print("Test accuracy：%.1f%%"%accuracy(test_prediction.eval(),test_labels))

Initialized
Minibatch loss at step 0：813.177368
Minibatch accuracy：7.8%
Validation accuracy：28.9%
Test accuracy：30.1%
Minibatch loss at step 500：225.934357
Minibatch accuracy：71.9%
Validation accuracy：79.3%
Test accuracy：80.1%
Minibatch loss at step 1000：119.349335
Minibatch accuracy：70.3%
Validation accuracy：79.3%
Test accuracy：80.8%
Minibatch loss at step 1500：69.114426
Minibatch accuracy：79.7%
Validation accuracy：81.9%
Test accuracy：84.5%
Minibatch loss at step 2000：41.441486
Minibatch accuracy：84.4%
Validation accuracy：83.6%
Test accuracy：86.9%
Minibatch loss at step 2500：25.273624
Minibatch accuracy：83.6%
Validation accuracy：84.3%
Test accuracy：88.4%
Minibatch loss at step 3000：15.578849
Minibatch accuracy：84.4%
Validation accuracy：85.0%
Test accuracy：89.2%
Minibatch loss at step 3500：9.683094
Minibatch accuracy：82.0%
Validation accuracy：85.3%
Test accuracy：90.2%
Minibatch loss at step 4000：6.046159
Minibatch accuracy：87.5%
Validation accuracy：85.7%
Test accuracy：90.7%
Minibatch l


##### Notes: This examples is from udacity tensorflow course.
#### Reference: https://classroom.udacity.com/courses/ud730/;  https://docs.pythontab.com/tensorflow/tutorials/mnist_pros/#_8
  